In [23]:
from hardwareModel import HardwareModel
import networkx as nx
import cvxpy as cp
import numpy as np

In [24]:
hw = HardwareModel(cfg="aladdin_const_with_mem")

netlist: [('Add0', {'type': 'pe', 'function': 'Add', 'in_use': 0, 'idx': 0}), ('Regs0', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 0}), ('Regs1', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 1}), ('Regs2', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 2}), ('Mult0', {'type': 'pe', 'function': 'Mult', 'in_use': 0, 'idx': 0}), ('Eq0', {'type': 'pe', 'function': 'Eq', 'in_use': 0, 'idx': 0}), ('Buf0', {'type': 'mem', 'function': 'Buf', 'in_use': 0, 'idx': 0, 'size': 22}), ('Buf1', {'type': 'mem', 'function': 'Buf', 'in_use': 0, 'idx': 1, 'size': 22}), ('Mem0', {'type': 'mem', 'function': 'MainMem', 'in_use': 0, 'idx': 0, 'size': 1024}), ('Regs3', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 3})]


In [25]:
print(hw.netlist)
print(hw.area)

DiGraph with 10 nodes and 38 edges
{'Add': 379348800.0, 'Sub': 379348800.0, 'And': 50369960.0, 'BitAnd': 50369960.0, 'BitOr': 50369960.0, 'BitXor': 50369960.0, 'Eq': 379348800.0, 'FloorDiv': 8340007000.0, 'Gt': 379348800.0, 'GtE': 379348800.0, 'Invert': 50369960.0, 'IsNot': 50369960.0, 'LShift': 24964610.0, 'Lt': 379348800.0, 'LtE': 379348800.0, 'Mod': 379348800.0, 'Mult': 8340007000.0, 'Not': 50369960.0, 'NotEq': 379348800.0, 'Or': 50369960.0, 'RShift': 24964610.0, 'Regs': 5981433.0, 'UAdd': 379348800.0, 'USub': 379348800.0, 'Buf': 0.0, 'MainMem': 5981433.0}


In [26]:
hw.area['Add']

379348800.0

In [27]:
digraph = hw.netlist
nodes = digraph.nodes
edges = digraph.edges
#print("Nodes:", list(nodes))
print("Edges:", list(edges))

Edges: [('Add0', 'Regs0'), ('Add0', 'Regs1'), ('Add0', 'Regs2'), ('Add0', 'Mult0'), ('Add0', 'Eq0'), ('Regs0', 'Regs1'), ('Regs0', 'Add0'), ('Regs0', 'Mult0'), ('Regs0', 'Eq0'), ('Regs0', 'Buf1'), ('Regs1', 'Regs0'), ('Regs1', 'Add0'), ('Regs1', 'Mult0'), ('Regs1', 'Eq0'), ('Regs1', 'Buf1'), ('Regs2', 'Regs3'), ('Regs2', 'Add0'), ('Regs2', 'Mult0'), ('Regs2', 'Eq0'), ('Regs2', 'Buf1'), ('Mult0', 'Add0'), ('Mult0', 'Regs0'), ('Mult0', 'Regs1'), ('Mult0', 'Regs2'), ('Mult0', 'Eq0'), ('Eq0', 'Add0'), ('Eq0', 'Mult0'), ('Eq0', 'Regs0'), ('Eq0', 'Regs1'), ('Eq0', 'Regs2'), ('Buf1', 'Regs3'), ('Buf1', 'Regs0'), ('Buf1', 'Mem0'), ('Buf1', 'Regs1'), ('Buf1', 'Regs2'), ('Mem0', 'Buf1'), ('Regs3', 'Regs2'), ('Regs3', 'Buf1')]


In [37]:
node_to_index = {node: i for i, node in enumerate(nodes)}
n = len(nodes) 
E = [(node_to_index[src], node_to_index[dst]) for src, dst in edges]

# Some fixed lij values for each edge (will slowly decrease the value until it doesn't work)
l = {edge: 10 for edge in E}

# Random weights for example, you may replace or adjust these
#w = np.random.randn(n)
w = np.abs(np.random.randn(n))
p = cp.Variable(n)
objective = cp.Maximize(cp.quad_form(p, np.diag(w)))
constraints = [cp.sum(cp.multiply(w, p)) == 0]  # Sum of w_i * p_i = 0

# Adding |p_i - p_j| <= l_ij constraints for each (i, j) in E
for (i, j) in E:
    constraints += [cp.abs(p[i] - p[j]) <= l[(i, j)]]

problem = cp.Problem(objective, constraints)
problem.solve()

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP, even though each sub-expression is.
You are trying to maximize a function that is convex.